In [176]:
import pandas as pd
import json
import gzip
import ast
import re
import numpy as np
import importlib
import validation

# Reload the functions module
importlib.reload(validation)

<module 'validation' from 's:\\Courses\\Data Science\\0. Henry\\M7 - Labs - Project MLOps _ steam\\game-recommendation-system\\data_transformation\\validation.py'>

## User reviews

### Creating a DataFrame

Creating a DataFrame from the australian_user_reviews JSON file

In [7]:
# Open JSON file and load data in a variable
review_data = []
with open(r"./raw_data/australian_user_reviews.json", encoding="MacRoman") as file:
    for line in file.readlines():
        review_data.append(ast.literal_eval(line))

In [8]:
# Create a DataFrame from variable
df_user_reviews = pd.DataFrame(review_data)
df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


### Preparing the user reviews data

#### Checking Null values

In [15]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


No null values in the dataset

#### Checking Duplicate values

In [11]:
# Call the function to check for duplicates in the "user_url" column
duplicates_user_url = validation.check_duplicates_summary(df_user_reviews, "user_url")

# Print the summary
print(duplicates_user_url)

                                      Duplicated Value  Occurrences
0    http://steamcommunity.com/profiles/76561198051...            3
2    http://steamcommunity.com/profiles/76561198045...            3
3    http://steamcommunity.com/profiles/76561198100...            3
4            http://steamcommunity.com/id/blablabla174            3
1    http://steamcommunity.com/profiles/76561198027...            3
..                                                 ...          ...
104    http://steamcommunity.com/id/syntheticflyingfox            2
103           http://steamcommunity.com/id/TandyTandon            2
102  http://steamcommunity.com/id/wasntthereanumber...            2
101               http://steamcommunity.com/id/norsktf            2
308         http://steamcommunity.com/id/OfficialvnmZz            2

[309 rows x 2 columns]


Duplicate user_url values are removed, as they will affect future measurements

#### Removing Duplicate values

In [14]:
df_user_reviews_1 = validation.remove_duplicates(df_user_reviews, "user_url")

Removed 314 duplicate rows based on 'user_url' column.


In [27]:
# Verifiying no duplicate values left

# Call the function to check for duplicates in the "user_id" column
duplicates_user_id = validation.check_duplicates_summary(df_user_reviews_1, "user_id")

# Print the summary
print(duplicates_user_id)


Empty DataFrame
Columns: [Duplicated Value, Occurrences]
Index: []


#### Checking None values

In [28]:
# Call the function to check for none values in the "user_id" column
none_user_id = validation.check_none_values(df_user_reviews_1, "user_id")

# Print the summary
print(none_user_id)

{'column_name': 'user_id', 'total_rows': 25485, 'num_none_values': 0}


## User Items

### Creating a DataFrame

Creating a DataFrame from the australian_users_items JSON file

In [35]:
# Open JSON file and load data in variable
items_data = []
with open(r"./raw_data/australian_users_items.json", encoding="MacRoman") as file:
    for line in file.readlines():
        items_data.append(ast.literal_eval(line))

In [37]:
# Create a DataFrame from variable
df_user_items = pd.DataFrame(items_data)
df_user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


### Preparing the users items data

#### Checking Null values

In [38]:
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


No Null values found

#### Checking Duplicate values

In [39]:
# Call the function to check for duplicates in the "user_url" column
duplicates_steam_id = validation.check_duplicates_summary(df_user_items, "steam_id")

# Print the summary
print(duplicates_steam_id)

      Duplicated Value  Occurrences
0    76561198094973305            3
6    76561198085989695            3
1    76561198072861800            3
9    76561198051777058            3
8    76561198081666970            3
..                 ...          ...
231  76561198026742195            2
232  76561198035336388            2
233  76561198071790027            2
234  76561198073621277            2
673  76561198086134170            2

[674 rows x 2 columns]


Duplicate steam_id values are removed, as they will affect future measurements

#### Removing Duplicate values

In [40]:
df_user_items_1 = validation.remove_duplicates(df_user_items, "steam_id")

Removed 685 duplicate rows based on 'steam_id' column.


In [41]:
# Verifiying no duplicate values left

# Call the function to check for duplicates in the "steam_id" column
duplicates_steam_id = validation.check_duplicates_summary(df_user_items_1, "steam_id")

# Print the summary
print(duplicates_steam_id)

Empty DataFrame
Columns: [Duplicated Value, Occurrences]
Index: []


#### Checking None values

In [42]:
# Call the function to check for none values in the "steam_id" column
none_steam_id = validation.check_none_values(df_user_items_1, "steam_id")

# Print the summary
print(none_steam_id)

{'column_name': 'steam_id', 'total_rows': 87625, 'num_none_values': 0}


## Steam Games

### Creating a DataFrame

Creating a DataFrame from the output_steam_games JSON file

In [139]:
# Open JSON file and load data in variable
games_data = []
with open("./raw_data/output_steam_games.json", encoding="MacRoman") as file:
    for line in file:
        games_data.append(json.loads(line))

In [140]:
# Create a DataFrame from variable
df_games_data = pd.DataFrame(games_data)
df_games_data

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


### Preparing the steam games data

#### Checking Duplicate values

In [141]:
# Call the function to check for duplicates in the "id" column
duplicates_games_id = validation.check_duplicates_summary(df_games_data, "id")

# Print the summary
print(duplicates_games_id)

  Duplicated Value  Occurrences
0           612880            2


Duplicate id values are removed, as they will affect future measurements

#### Removing Duplicate values

In [142]:
df_games_data_1 = validation.remove_duplicates(df_games_data, "id")

Removed 88312 duplicate rows based on 'id' column.


In [143]:
# Verifiying no duplicate values left

# Call the function to check for duplicates in the "id" column
duplicates_games_id = validation.check_duplicates_summary(df_games_data_1, "id")

# Print the summary
print(duplicates_games_id)

Empty DataFrame
Columns: [Duplicated Value, Occurrences]
Index: []


In [144]:
df_games_data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24081 non-null  object
 1   genres        28850 non-null  object
 2   app_name      32131 non-null  object
 3   title         30083 non-null  object
 4   url           32132 non-null  object
 5   release_date  30066 non-null  object
 6   tags          31970 non-null  object
 7   reviews_url   32132 non-null  object
 8   specs         31463 non-null  object
 9   price         30755 non-null  object
 10  early_access  32132 non-null  object
 11  id            32132 non-null  object
 12  developer     28834 non-null  object
dtypes: object(13)
memory usage: 3.4+ MB


#### Checking None values

In [145]:
# Call the function to check for none values in the "id" column
#none_games_id = validation.check_none_values(df_games_data_1, "id")

# check_none_values(dataframe, column_name, check_other_columns=False, min_none_null_values=4):
none_games_id = validation.check_none_values(df_games_data_1, "id", True, 4)

# Print the summary
print(none_games_id)

{'column_name': 'id', 'total_rows': 32133, 'num_none_values': 1, 'rows_below_threshold': 2062}


#### Removing None values

In [146]:
# Call the function to remove none values in the "id" column
df_games_data_2 = validation.remove_none_values(df_games_data_1, "id", True, 4)

In [147]:
# Verifiying no none values left

# Call the function to check for duplicates in the "id" column
none_games_id = validation.check_none_values(df_games_data_2, "id", True, 4)

# Print the summary
print(none_games_id)

{'column_name': 'id', 'total_rows': 30071, 'num_none_values': 0, 'rows_below_threshold': 0}


In [148]:
df_games_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30071 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24081 non-null  object
 1   genres        28849 non-null  object
 2   app_name      30071 non-null  object
 3   title         30071 non-null  object
 4   url           30071 non-null  object
 5   release_date  30053 non-null  object
 6   tags          29918 non-null  object
 7   reviews_url   30071 non-null  object
 8   specs         29413 non-null  object
 9   price         28820 non-null  object
 10  early_access  30071 non-null  object
 11  id            30071 non-null  object
 12  developer     28834 non-null  object
dtypes: object(13)
memory usage: 3.2+ MB


#### Transforming to numeric type

To have better performance in the API

##### id column

In [149]:
id_summary = column_data_types_summary(df_games_data_2, "id")
id_summary

{'column_name': 'id', 'data_type_counts': {"<class 'str'>": 30071}}

In [155]:
df_games_data_3 = df_games_data_2.copy()

In [156]:
id_transformation = transform_to_numeric(df_games_data_3, "id")
id_transformation

{'total_values': 30071,
 'numeric_count': 30071,
 'non_numeric_count': 0,
 'stored_info': []}

In [177]:
price_summary = column_data_types_summary(df_games_data_3, "price")
price_summary

{'column_name': 'price',
 'data_type_counts': {"<class 'float'>": 28492,
  "<class 'int'>": 1556,
  "<class 'str'>": 23}}

In [178]:
price_transformation = transform_to_numeric(df_games_data_3, "price")
price_transformation

{'total_values': 30071,
 'numeric_count': 28797,
 'non_numeric_count': 23,
 'stored_info': [(89230, 'Free Demo'),
  (89231, 'Free Demo'),
  (89279, 'Play for Free!'),
  (89291, 'Play for Free!'),
  (90715, 'Install Now'),
  (91181, 'Play WARMACHINE: Tactics Demo'),
  (91624, 'Free Mod'),
  (92142, 'Install Theme'),
  (92228, 'Third-party'),
  (92336, 'Play Now'),
  (93899, 'Free HITMAN™ Holiday Pack'),
  (102257, 'Free Mod'),
  (111044, 'Play the Demo'),
  (113310, 'Starting at $499.00'),
  (113311, 'Starting at $449.00'),
  (114527, 'Play Now'),
  (115214, 'Free to Try'),
  (116624, 'Free Mod'),
  (116625, 'Free Mod'),
  (117676, 'Free Movie'),
  (118690, 'Free to Use'),
  (119220, 'Free Demo'),
  (120148, 'Third-party')]}

In [160]:
df_games_data_3["price"][89152]

'Free to Play'

In [174]:
# Convert "free to play" and "free" to numeric
df_games_data_3["price"] = np.where(
    df_games_data_3["price"].str.lower().isin(["free to play", "free", "Free Demo", "Free to Use", "Free Mod", "Free to try", "Install Theme", "Play the Demo", "Play for Free!", "Install Now", "Play WARMACHINE: Tactics Demo", "Third-party", "play now", "Free HITMAN™ Holiday Pack"]), 0, df_games_data_3["price"]
)

In [164]:
df_games_data_3["price"][89230]

'Free Demo'